In [1]:
import os 
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.utils import to_categorical

# 新段落

# 新段落

In [2]:
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import InputLayer
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.applications.inception_resnet_v2 import preprocess_input

In [3]:
input_shape = (128, 128, 3)
num_classes = 2

image_resize = 128

batch_size_training = 100
batch_size_validation = 100

In [ ]:
googleNet_model = InceptionResNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
googleNet_model.trainable = True
model = Sequential()
model.add(googleNet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=2, activation='softmax'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
model.summary()

In [4]:
data_generator = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    validation_split=0.2
)

In [5]:
train_generator = data_generator.flow_from_directory(
    'Images1',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical',
    subset='training')

val_generator = data_generator.flow_from_directory(
    'Images1',
    target_size=(image_resize, image_resize),
    batch_size=batch_size_training,
    class_mode='categorical',
    subset='validation'
)



Found 135077 images belonging to 2 classes.
Found 33768 images belonging to 2 classes.


In [6]:
googleNet_model = InceptionResNetV2(include_top=False, weights='imagenet',input_shape=input_shape)
googleNet_model.trainable = True
model = Sequential()
model.add(googleNet_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(units=2, activation='softmax'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False),
              metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Model)  (None, 2, 2, 1536)        54336736  
_________________________________________________________________
global_average_pooling2d (Gl (None, 1536)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 3074      
Total params: 54,339,810
Trainable params: 54,279,266
Non-trainable params: 60,544
_________________________________________________________________


In [7]:
early_stopping = EarlyStopping(monitor='val_loss',
                min_delta=0,
                patience=2,
                verbose=0, mode='auto')

In [8]:
steps_per_epoch_training = len(train_generator)
steps_per_epoch_validation = len(val_generator)
num_epochs = 20

In [13]:
from tensorflow.keras.callbacks import *
filepath = "mycnn/epochs:{epoch:03d}-val_acc:{val_acc:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [14]:
fit_history = model.fit_generator(
    train_generator,
    steps_per_epoch=steps_per_epoch_training,
    epochs=num_epochs,
    validation_data=val_generator,
    validation_steps=steps_per_epoch_validation,
    verbose=1,
    callbacks=callbacks_list
)

Epoch 1/20
1350/1351 [============================>.] - ETA: 0s - loss: 0.1753 - acc: 0.9230
Epoch 00001: val_acc improved from -inf to 0.87385, saving model to mycnn/epochs:001-val_acc:0.874.hdf5
1351/1351 [==============================] - 678s 502ms/step - loss: 0.1753 - acc: 0.9230 - val_loss: 0.3046 - val_acc: 0.8738
Epoch 2/20
1350/1351 [============================>.] - ETA: 0s - loss: 0.0995 - acc: 0.9570
Epoch 00002: val_acc improved from 0.87385 to 0.87695, saving model to mycnn/epochs:002-val_acc:0.877.hdf5
1351/1351 [==============================] - 672s 497ms/step - loss: 0.0995 - acc: 0.9570 - val_loss: 0.3518 - val_acc: 0.8770
Epoch 3/20
1350/1351 [============================>.] - ETA: 0s - loss: 0.0613 - acc: 0.9739
Epoch 00003: val_acc improved from 0.87695 to 0.87870, saving model to mycnn/epochs:003-val_acc:0.879.hdf5
1351/1351 [==============================] - 674s 499ms/step - loss: 0.0613 - acc: 0.9739 - val_loss: 0.4083 - val_acc: 0.8787
Epoch 4/20
1350/1351 [

In [15]:
model.save('deepfake-detection-model.h5')

In [16]:
import matplotlib.pyplot as plt

In [20]:
import json
with open('myfile.json', 'w') as file:

    json.dump(fit_history.history, file)

In [23]:
from tensorflow.keras.models import load_model

In [24]:
model2 = load_model('C:\Users\KaiY\Documents\DeepFake\')

In [ ]:
input_shape = (128, 128, 3)
data_dir = 'dataset'
data = [f for f in os.listdir(data_dir) if f.endswith('.png')]
cap = cv2.VideoCapture('test_videos/jzmzdispyo.mp4')